# Data Cleansing and Preparation

In [1]:
import numpy as np
import pandas as pd
import time
import datetime

In [2]:
for i in os.listdir('../datasets/raw'):
    print(i)

databoks-aug-2022.xlsx
kemenaker-2018-2022.xlsx
upah-minimum-2023.xlsx
upah-minimum-provinsi-raw-bps.csv


There's multiple dataset from multiple source with different format that need to be processed into a single dataset. First, let's see the data from each of the datasets.

In [3]:
# load dataframes
df1 = pd.read_excel("../datasets/raw/databoks-aug-2022.xlsx")
df2 = pd.read_excel("../datasets/raw/kemenaker-2018-2022.xlsx")
df3 = pd.read_excel("../datasets/raw/upah-minimum-2023.xlsx")
df4 = pd.read_csv("../datasets/raw/upah-minimum-provinsi-raw-bps.csv", delimiter=";")

# display shape
print("df1's shape: ", df1.shape)
print("df2's shape: ", df2.shape)
print("df3's shape: ", df3.shape)
print("df4's shape: ", df4.shape)

df1's shape:  (35, 2)
df2's shape:  (34, 6)
df3's shape:  (34, 2)
df4's shape:  (34, 21)


In [4]:
# display the first five rows for each dataframe
display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())

,nama_data,2022
0,DKI Jakarta,4641854
1,Papua,3561932
2,Sulawesi Utara,3310723
3,Kep. Bangka Belitung,3264884
4,Papua Barat,3200000


,Provinsi,2018,2019,2020,2021,2022*
0,Aceh,2700000.00,2916810.00,3165031.00,3165031.00,3166460.00
1,Sumatera Utara,2132188.68,2303403.43,2499423.06,2499423.06,2522609.94
2,Sumatera Barat,2119067.00,2289220.00,2484041.00,2484041.00,2512539.00
3,Riau,2464154.06,2662025.63,2888584.01,2888564.01,2938564.01
4,Jambi,2243718.56,2423889.16,2630162.13,2630162.13,2698940.87


,nama_data,2023
0,DKI Jakarta,4901798
1,Papua,3864696
2,Kep. Bangka Belitung,3498479
3,Sulawesi Utara,3485000
4,Aceh,3413666


,provinsi,2020,2019,2018,2016,2015,2014,2013,2012,2011,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,ACEH,3165031,2916810,2700000,2118500,1900000,1750000,1550000,1400000,1350000,...,1200000,1000000,850000,820000,620000,550000,425000,330000,300000,265000
1,SUMATERA UTARA,2499423,2303403,2132189,1811875,1625000,1505850,1375000,1200000,1035500,...,905000,822205,761000,737794,600000,537000,505000,464000,340500,254000
2,SUMATERA BARAT,2484041,2289220,2119067,1800725,1615000,1490000,1350000,1150000,1055000,...,880000,800000,725000,650000,540000,480000,435000,385000,250000,200000
3,RIAU,2888564,2662026,2464154,2095000,1878000,1700000,1400000,1238000,1120000,...,901600,800000,710000,637000,551500,476900,437500,394000,329000,250700
4,JAMBI,2630162,2423889,2243719,1906650,1710000,1502300,1300000,1142500,1028000,...,800000,724000,658000,563000,485000,425000,390000,304000,245000,173000


As we can see, the dataframes contain the provinces name and its minimum wage for every year. After briefly analyzing the dataframes, there's several task that needs to be done in order to merged all of these dataframes into a single dataframe. Here's the to-do task:
- The 2022 minimum wage in `df2` its only temporary (Feb 2022) and it needs to be replaced by the minimum wage in `df1` since its the latest update (Aug 2022).
- Add the 2023 minimum wage from `df3` to merged dataframe of `df1` and `df2`.
- Add 2023, 2022, and 2021 minimum wage from merged dataframe before to `df4`. 
- Reformat the final merged dataframe into a long format dataframe using `pd.melt()`.
- Export the long dataframe so it can use for data analyzing and built the data visualization.

## A. Merge DF1 and DF2

Check the provinces name from each dataframe

In [5]:
df1["nama_data"].unique()

array(['DKI Jakarta', 'Papua', 'Sulawesi Utara', 'Kep. Bangka Belitung',
       'Papua Barat', 'Aceh', 'Sulawesi Selatan', 'Sumatra Selatan',
       'Kep. Riau', 'Kalimantan Utara', 'Kalimantan Timur', 'Riau',
       'Kalimantan Tengah', 'Kalimantan Selatan', 'Maluku Utara',
       'Gorontalo', 'Rata-rata Nasional', 'Sulawesi Tenggara', 'Jambi',
       'Sulawesi Barat', 'Maluku', 'Sumatra Utara', 'Bali',
       'Sumatra Barat', 'Banten', 'Lampung', 'Kalimantan Barat',
       'Sulawesi Tengah', 'Bengkulu', 'Nusa Tenggara Barat',
       'Nusa Tenggara Timur', 'Jawa Timur', 'Jawa Barat', 'DI Yogyakarta',
       'Jawa Tengah'], dtype=object)

In [6]:
df2["Provinsi"].unique()

array(['Aceh', 'Sumatera Utara', 'Sumatera Barat', 'Riau', 'Jambi',
       'Sumatera Selatan', 'Bengkulu', 'Lampung', 'Bangka Belitung',
       'Kepulauan Riau', 'DKI Jakarta', 'Jawa Barat', 'Jawa Tengah',
       'DI. Yogyakarta', 'Jawa Timur', 'Banten', 'Bali',
       'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Kalimantan Barat',
       'Kalimantan Tengah', 'Kalimantan Selatan', 'Kalimantan Timur',
       'Kalimantan Utara', 'Sulawesi Utara', 'Sulawesi Tengah',
       'Sulawesi Selatan', 'Sulawesi Tenggara', 'Gorontalo',
       'Sulawesi Barat', 'Maluku', 'Maluku Utara', 'Papua Barat', 'Papua'],
      dtype=object)

Rename `df1` first column to `Provinsi` so it will have the same column name with `df2`. Filter which province that doesn't have the same name to provinces name in `df2`.

In [7]:
# rename column
df1 = df1.rename(columns={"nama_data":"Provinsi"})

# merge the dataframes and display what provinces that only contain in the left and right dataframe
right_merged = df2.merge(df1, on="Provinsi", how="right", indicator=True)
left_merged = df2.merge(df1, on="Provinsi", how="left", indicator=True)

# filter
right_merged = right_merged[right_merged["_merge"]=="right_only"]
left_merged = left_merged[left_merged["_merge"]=="left_only"]

# display the fitlered data
print("left only: ", left_merged["Provinsi"].unique())
print("right only: ", right_merged["Provinsi"].unique())

left only:  ['Sumatera Utara' 'Sumatera Barat' 'Sumatera Selatan' 'Bangka Belitung'
 'Kepulauan Riau' 'DI. Yogyakarta']
right only:  ['Kep. Bangka Belitung' 'Sumatra Selatan' 'Kep. Riau' 'Rata-rata Nasional'
 'Sumatra Utara' 'Sumatra Barat' 'DI Yogyakarta']


Rename provinces name in `df1` (right only) to `df2` provinces name. Exlcude "Rata-rata Nasional" since it's the aggregation of national minimum wage.

In [8]:
# rename the provinces name
df1["Provinsi"] = df1["Provinsi"].replace({'Kep. Bangka Belitung':'Bangka Belitung', 'Sumatra Selatan':'Sumatera Selatan',
                                          'Kep. Riau':'Kepulauan Riau','Sumatra Utara':'Sumatera Utara',
                                           'Sumatra Barat':'Sumatera Barat', 'DI Yogyakarta':'DI. Yogyakarta'})

# merge dataframes
merged_df = df2.merge(df1, on="Provinsi", how="outer")

# exclude neccesary record
merged_df = merged_df[(merged_df["Provinsi"]=="Rata-rata Nasional")==False]

# take only 2022 and 2021 column
merged_df = merged_df[["Provinsi", 2022, 2021]]

# display the result
merged_df

,Provinsi,2022,2021
0,Aceh,3166460,3165031.00
1,Sumatera Utara,2522610,2499423.06
2,Sumatera Barat,2512539,2484041.00
3,Riau,2938564,2888564.01
4,Jambi,2698941,2630162.13
5,Sumatera Selatan,3144446,3144446.00
6,Bengkulu,2238094,2215000.00
7,Lampung,2440486,2432001.57
8,Bangka Belitung,3264884,3230023.66
9,Kepulauan Riau,3050172,3005460.00


## B. Add 2023 Minimum Wage from DF3

We already have first merged dataframe of `df1` and `df2` in the `merged_df`. Now, we need to add 2023 minimum wage from `df3`.

In [9]:
df3.head(2)

,nama_data,2023
0,DKI Jakarta,4901798
1,Papua,3864696


In [10]:
# rename df3 column
df3 = df3.rename(columns={"nama_data":"Provinsi"})

# merge the dataframes and display what provinces that only contain in the left and right dataframe
right_merged = merged_df.merge(df3, on="Provinsi", how="right", indicator=True)
left_merged = merged_df.merge(df3, on="Provinsi", how="left", indicator=True)

# filter
right_merged = right_merged[right_merged["_merge"]=="right_only"]
left_merged = left_merged[left_merged["_merge"]=="left_only"]

# display the fitlered data
print("left only: ", left_merged["Provinsi"].unique())
print("right only: ", right_merged["Provinsi"].unique())

left only:  ['Aceh' 'Sumatera Utara' 'Sumatera Barat' 'Riau' 'Jambi'
 'Sumatera Selatan' 'Bengkulu' 'Lampung' 'Bangka Belitung'
 'Kepulauan Riau' 'DKI Jakarta' 'Jawa Barat' 'Jawa Tengah'
 'DI. Yogyakarta' 'Jawa Timur' 'Banten' 'Bali' 'Nusa Tenggara Barat'
 'Nusa Tenggara Timur' 'Kalimantan Barat' 'Kalimantan Tengah'
 'Kalimantan Selatan' 'Kalimantan Timur' 'Kalimantan Utara'
 'Sulawesi Utara' 'Sulawesi Tengah' 'Sulawesi Selatan' 'Sulawesi Tenggara'
 'Gorontalo' 'Sulawesi Barat' 'Maluku' 'Maluku Utara' 'Papua Barat'
 'Papua']
right only:  [' DKI Jakarta ' ' Papua ' ' Kep. Bangka Belitung ' ' Sulawesi Utara '
 ' Aceh ' ' Sumatera Selatan ' ' Sulawesi Selatan ' ' Papua Barat '
 ' Kep. Riau ' ' Kalimantan Utara ' ' Kalimantan Timur ' ' Riau '
 ' Kalimantan Tengah ' ' Kalimantan Selatan ' ' Gorontalo ' ' Maluku '
 ' Maluku Utara ' ' Jambi ' ' Sulawesi Barat ' ' Sulawesi Tenggara '
 ' Sumatera Barat ' ' Bali ' ' Sumatera Utara ' ' Banten ' ' Lampung '
 ' Kalimantan Barat ' ' Sulawesi Tengah 

For each name of the provinces in the `df3`, there's leading and trailing whitespace. We need to remove it first before we now what province that need to be rename so it match the name of provinces in `merged_df`.

In [11]:
# remove leading and trailing whitespace
df3["Provinsi"] = df3["Provinsi"].str.rstrip()
df3["Provinsi"] = df3["Provinsi"].str.lstrip()

# merge the dataframes and display what provinces that only contain in the left and right dataframe
right_merged = merged_df.merge(df3, on="Provinsi", how="right", indicator=True)
left_merged = merged_df.merge(df3, on="Provinsi", how="left", indicator=True)

# filter
right_merged = right_merged[right_merged["_merge"]=="right_only"]
left_merged = left_merged[left_merged["_merge"]=="left_only"]

# display the fitlered data
print("left only: ", left_merged["Provinsi"].unique())
print("right only: ", right_merged["Provinsi"].unique())

left only:  ['Bangka Belitung' 'Kepulauan Riau' 'DI. Yogyakarta']
right only:  ['Kep. Bangka Belitung' 'Kep. Riau' 'DI Yogyakarta']


In [12]:
# rename the provinces name
df3["Provinsi"] = df3["Provinsi"].replace({'Kep. Bangka Belitung':'Bangka Belitung',
                                          'Kep. Riau':'Kepulauan Riau','DI Yogyakarta':'DI. Yogyakarta'})

# merge dataframes
merged_df2 = merged_df.merge(df3, on="Provinsi", how="outer")

# display the result
merged_df2

,Provinsi,2022,2021,2023
0,Aceh,3166460,3165031.00,3413666
1,Sumatera Utara,2522610,2499423.06,2710493
2,Sumatera Barat,2512539,2484041.00,2742476
3,Riau,2938564,2888564.01,3191662
4,Jambi,2698941,2630162.13,2943000
5,Sumatera Selatan,3144446,3144446.00,3404177
6,Bengkulu,2238094,2215000.00,2418280
7,Lampung,2440486,2432001.57,2633284
8,Bangka Belitung,3264884,3230023.66,3498479
9,Kepulauan Riau,3050172,3005460.00,3279194


## C. Merge with DF4

In [13]:
# display first five rows of the df4

display(df4.head())

,provinsi,2020,2019,2018,2016,2015,2014,2013,2012,2011,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,ACEH,3165031,2916810,2700000,2118500,1900000,1750000,1550000,1400000,1350000,...,1200000,1000000,850000,820000,620000,550000,425000,330000,300000,265000
1,SUMATERA UTARA,2499423,2303403,2132189,1811875,1625000,1505850,1375000,1200000,1035500,...,905000,822205,761000,737794,600000,537000,505000,464000,340500,254000
2,SUMATERA BARAT,2484041,2289220,2119067,1800725,1615000,1490000,1350000,1150000,1055000,...,880000,800000,725000,650000,540000,480000,435000,385000,250000,200000
3,RIAU,2888564,2662026,2464154,2095000,1878000,1700000,1400000,1238000,1120000,...,901600,800000,710000,637000,551500,476900,437500,394000,329000,250700
4,JAMBI,2630162,2423889,2243719,1906650,1710000,1502300,1300000,1142500,1028000,...,800000,724000,658000,563000,485000,425000,390000,304000,245000,173000


Rename `provinsi` column, and change the provinces name into a title case.

In [14]:
# rename column
df4 = df4.rename(columns={"provinsi":"Provinsi"})

# change name to title cas
df4["Provinsi"] = df4["Provinsi"].str.title()

# display its first five provinces
df4["Provinsi"].head()

0              Aceh
1    Sumatera Utara
2    Sumatera Barat
3              Riau
4             Jambi
Name: Provinsi, dtype: object

Match the provinces name by fiter it.

In [15]:
right_merged = merged_df2.merge(df4, on="Provinsi", how="right", indicator=True)
left_merged = merged_df2.merge(df4, on="Provinsi", how="left", indicator=True)

# filter
right_merged = right_merged[right_merged["_merge"]=="right_only"]
left_merged = left_merged[left_merged["_merge"]=="left_only"]

# display result
print("left only: ", left_merged["Provinsi"].unique())
print("right only: ", right_merged["Provinsi"].unique())

left only:  ['Bangka Belitung' 'Kepulauan Riau' 'DKI Jakarta' 'DI. Yogyakarta']
right only:  ['Kep. Bangka Belitung' 'Kep. Riau' 'Dki Jakarta' 'Di Yogyakarta']


In [16]:
# replace the provinces name so it will match to the toher dataframe
df4["Provinsi"] = df4["Provinsi"].replace({'Kep. Bangka Belitung':'Bangka Belitung', 'Dki Jakarta':'DKI Jakarta',
                                          'Kep. Riau':'Kepulauan Riau','Di Yogyakarta':'DI. Yogyakarta'})

# final merge
merged_all = merged_df2.merge(df4, on="Provinsi", how="outer")

# display the result
merged_all

,Provinsi,2022,2021,2023,2020,2019,2018,2016,2015,2014,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,Aceh,3166460,3165031.00,3413666,3165031,2916810,2700000,2118500,1900000,1750000,...,1200000,1000000,850000,820000,620000,550000,425000,330000,300000,265000
1,Sumatera Utara,2522610,2499423.06,2710493,2499423,2303403,2132189,1811875,1625000,1505850,...,905000,822205,761000,737794,600000,537000,505000,464000,340500,254000
2,Sumatera Barat,2512539,2484041.00,2742476,2484041,2289220,2119067,1800725,1615000,1490000,...,880000,800000,725000,650000,540000,480000,435000,385000,250000,200000
3,Riau,2938564,2888564.01,3191662,2888564,2662026,2464154,2095000,1878000,1700000,...,901600,800000,710000,637000,551500,476900,437500,394000,329000,250700
4,Jambi,2698941,2630162.13,2943000,2630162,2423889,2243719,1906650,1710000,1502300,...,800000,724000,658000,563000,485000,425000,390000,304000,245000,173000
5,Sumatera Selatan,3144446,3144446.00,3404177,3043111,2804453,2595995,2206000,1974346,1825000,...,824730,743000,662000,604000,503700,460000,403500,331500,255000,190000
6,Bengkulu,2238094,2215000.00,2418280,2213604,2040407,1888741,1605000,1500000,1350000,...,735000,690000,644838,516000,430000,363000,330000,295000,240000,173000
7,Lampung,2440486,2432001.57,2633284,2432002,2241270,2074673,1763000,1581000,1399037,...,691000,617000,555000,505000,405000,377500,350000,310000,240000,192000
8,Bangka Belitung,3264884,3230023.66,3498479,3230024,2976706,2755444,2341500,2100000,1640000,...,850000,813000,720000,640000,560000,447900,379500,345000,255000,190000
9,Kepulauan Riau,3050172,3005460.00,3279194,3005460,2769754,2563875,2178710,1954000,1665000,...,892000,833000,805000,760000,557000,-,-,-,421500,300000


Looks good. We have 34 provinces and minimum wage records from 2000 to 2023. Now we can change the dataframe format into long format dataframe.

## D. Reformat the Dataframe

In [17]:
# change to long format
melted_df = pd.melt(merged_all, id_vars=["Provinsi"], value_vars=merged_all.columns[1:], var_name="Tahun", value_name="UpahMinimumProvinsi")

# display the result
melted_df

,Provinsi,Tahun,UpahMinimumProvinsi
0,Aceh,2022,3166460
1,Sumatera Utara,2022,2522610
2,Sumatera Barat,2022,2512539
3,Riau,2022,2938564
4,Jambi,2022,2698941
...,...,...,...
777,Sulawesi Barat,2000,-
778,Maluku,2000,180000
779,Maluku Utara,2000,180000
780,Papua Barat,2000,-


In [18]:
# display the dataframe information
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Provinsi             782 non-null    object
 1   Tahun                782 non-null    object
 2   UpahMinimumProvinsi  782 non-null    object
dtypes: object(3)
memory usage: 18.5+ KB


In [19]:
# change datatype so we can sort it
melted_df['Tahun'] = melted_df['Tahun'].astype('int')

# sort the dataframe
melted_df.sort_values(by="Tahun", inplace=True)

# reset index
melted_df.reset_index(drop=True, inplace=True)

# display the result
melted_df

,Provinsi,Tahun,UpahMinimumProvinsi
0,Papua,2000,315000
1,DI. Yogyakarta,2000,194500
2,Jawa Tengah,2000,185000
3,Jawa Barat,2000,230000
4,DKI Jakarta,2000,286000
...,...,...,...
777,DI. Yogyakarta,2023,2025046
778,Jawa Timur,2023,2040244
779,Banten,2023,2661280
780,Kepulauan Riau,2023,3279194


In [20]:
# display the dataframe information
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Provinsi             782 non-null    object
 1   Tahun                782 non-null    int32 
 2   UpahMinimumProvinsi  782 non-null    object
dtypes: int32(1), object(2)
memory usage: 15.4+ KB


In [21]:
melted_df[melted_df["UpahMinimumProvinsi"]=="-"]

,Provinsi,Tahun,UpahMinimumProvinsi
17,Papua Barat,2000,-
21,Sulawesi Barat,2000,-
27,Kalimantan Utara,2000,-
52,Papua Barat,2001,-
55,Sulawesi Barat,2001,-
61,Kalimantan Utara,2001,-
72,Kepulauan Riau,2002,-
87,Papua Barat,2002,-
90,Sulawesi Barat,2002,-
95,Kalimantan Utara,2002,-


Replace "-" to 0 so we able to rounding the minimum wage column.

In [22]:
# replace "-" values
melted_df["UpahMinimumProvinsi"] = melted_df["UpahMinimumProvinsi"].replace({"-":0})

# rounding by turn to int
melted_df["UpahMinimumProvinsi"] = melted_df["UpahMinimumProvinsi"].astype('int')

# display 
melted_df

,Provinsi,Tahun,UpahMinimumProvinsi
0,Papua,2000,315000
1,DI. Yogyakarta,2000,194500
2,Jawa Tengah,2000,185000
3,Jawa Barat,2000,230000
4,DKI Jakarta,2000,286000
...,...,...,...
777,DI. Yogyakarta,2023,2025046
778,Jawa Timur,2023,2040244
779,Banten,2023,2661280
780,Kepulauan Riau,2023,3279194


In [23]:
melted_df[melted_df["UpahMinimumProvinsi"]=="-"]

,Provinsi,Tahun,UpahMinimumProvinsi


In [24]:
# take sample from 2019
melted_df[melted_df["Tahun"]==2019]

,Provinsi,Tahun,UpahMinimumProvinsi
612,DI. Yogyakarta,2019,1570923
613,Jawa Tengah,2019,1605396
614,Jawa Barat,2019,1668373
615,DKI Jakarta,2019,3940973
616,Kepulauan Riau,2019,2769754
617,Bangka Belitung,2019,2976706
618,Lampung,2019,2241270
619,Riau,2019,2662026
620,Sumatera Selatan,2019,2804453
621,Jambi,2019,2423889


Ok, everything looks good. We can save the dataframe to csv so we can use it later.

In [25]:
melted_df.to_csv("../datasets/processed/UMP-long-format.csv", index=False)